In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from statsmodels.tsa.seasonal import seasonal_decompose
import matplotlib.pyplot as plt
import plotly.express as px
import lightgbm as lgb
from catboost import CatBoostRegressor
!pip install optuna
import optuna  
from optuna.integration import LightGBMPruningCallback



In [ ]:
data = pd.read_csv('taxi.csv', index_col=[0], parse_dates=[0])
data.sort_index(inplace=True)
data = data.resample('1H').sum()

In [ ]:
data.info()

No missing values

In [ ]:
data.head()

Starts from 2018-03-01 00:00:00

In [ ]:
data.tail()

End at 2018-08-31 23:00:00

Let's break the orders into one contribute to the trend, seasonality and general noise 

In [ ]:
decomposed = seasonal_decompose(data)

plt.figure(figsize=(6, 8))
plt.subplot(311)
# To display the graph correctly, specify its
# axes ax equal to plt.gca() (gca = get current axis)
decomposed.trend.plot(ax=plt.gca())
plt.title('Trend')
plt.subplot(312)
decomposed.seasonal.plot(ax=plt.gca())
plt.title('Seasonality')
plt.subplot(313)
decomposed.resid.plot(ax=plt.gca())
plt.title('Residuals')
plt.tight_layout()

There is a obvious trend of increasing number of order in this period. The residuals stays almost the same. With Seasonality we have problem with this time sample. I guess that by sampeling shorter period we might learn more. I will take half month  

In [ ]:
new_data = decomposed.seasonal['2018-05-01':'2018-05-15']
new_data.plot()

Now it looks more obvious. I will try 3 days to make it clearer

In [ ]:
new_data = decomposed.seasonal['2018-05-01':'2018-05-03']
px.line(new_data).show()

We can see the highest peak of orders is at midnight. Second high peak is afternoon by 4 PM. Then at 10 AM . And the lowest time is 6 AM

Now let's check the rolling mean and std of orders during this time

In [ ]:
new_data = data.copy(deep=True)
new_data['mean'] = data['num_orders'].rolling(50).mean()
new_data['std'] = data['num_orders'].rolling(50).std()
new_data.plot() 

By the end of this period in august we can see an increase in orders amount and also higher variance

In [ ]:
# Creating features
def make_features(df, max_lag, rolling_mean_size_hour, rolling_mean_size_day):
    column_heading = df.columns[0] 
    df['month'] = df.index.month
    df['day'] = df.index.day
    df['dayofweek'] = df.index.dayofweek
    df['hour'] = df.index.hour

    for lag in range(1, max_lag + 1):
        df['lag_{}'.format(lag)] = df[column_heading].shift(lag)

    df['rolling_mean_hour'] = df[column_heading].shift().rolling(rolling_mean_size_hour).mean()
    df['rolling_mean_day'] = df[column_heading].shift().rolling(rolling_mean_size_day).mean()

make_features(data, 5, 2, 24)
data

In [ ]:
# remove the nan cause by lag and rolling mean in the dataset beggining
data.dropna(inplace=True)

In [ ]:
# Split train and test 
X = data.drop(['num_orders'], axis=1)
y = data[['num_orders']]

X_train, X_test, y_train, y_test = train_test_split(X, y, shuffle=False, test_size=0.1)
X_train.shape, X_test.shape, y_train.shape, y_test.shape


ok

Linear Regression

In [ ]:
lr = LinearRegression()
lr.fit(X_train, y_train)

predict = lr.predict(X_test)

print("RMSE for the test set: ", mean_squared_error(y_test, predict)**0.5)

For simple Linear Regression the target RMSE of 48 not achieved

Let's try light GBM

# laoding data
lgb_train = lgb.Dataset(X_train, y_train)
lgb_eval = lgb.Dataset(X_test, y_test, reference=lgb_train)

In [ ]:
%%time

lgbm_regressor = lgb.LGBMRegressor(
    boosting_type='gbdt',
    learning_rate=0.1,
    n_estimators=10000,
    random_state=12345
)

lgbm_regressor.fit(
    X_train,
    y_train,
    eval_set=[(X_test, y_test)],
    eval_metric='RMSE',
    callbacks=[lgb.early_stopping(stopping_rounds=100)]
)

y_predict = lgbm_regressor.predict(X_test)
print(f"LightGBM RMSE on the test set: {lgbm_regressor.best_score_['valid_0']['rmse']:.2f}")

With light GBM the RMSE is lower then 48

Now let's do the same but this time optimise the parameters: learning_rate, num_leaves and max_depth

In [ ]:
def objective(trial, X, y):
    param_grid = {
        #         "device_type": trial.suggest_categorical("device_type", ['gpu']),
        "n_estimators": trial.suggest_categorical("n_estimators", [10000]),
        "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.3),
        "num_leaves": trial.suggest_int("num_leaves", 20, 3000, step=20),
        "max_depth": trial.suggest_int("max_depth", 3, 12),
    }
    
    model = lgb.LGBMRegressor(**param_grid)
    model.fit(
            X_train,
            y_train,
            eval_set=[(X_test, y_test)],
            eval_metric="rmse",
            early_stopping_rounds=100,
            callbacks=[lgb.early_stopping(stopping_rounds=100)]  # Add a pruning callback
        )
    preds = model.predict(X_test)
    score = mean_squared_error(y_test, preds)**0.5
        
    return score

In [ ]:
%%time

study = optuna.create_study(direction="minimize", study_name="LGBM regressor")
func = lambda trial: objective(trial, X, y)
study.optimize(func, n_trials=20)

In [ ]:
print(f"\tBest value (rmse): {study.best_value:.5f}")
print(f"\tBest value (rmse): {study.best_value:.5f}")
print(f"\tBest params:")

for key, value in study.best_params.items():
    print(f"\t\t{key}: {value}")

I managed to squeezed up a bit the RMSE score and now it's almost 42! 

For conclusion the hyperparameter tuning on the lgbm yield the best results. This together with optimizing the features I managed to get a good RMSE, much below the requirements